In [2]:
import itertools
!pip install pymorphy2[fast]
from pymorphy2 import MorphAnalyzer
from collections import Counter, defaultdict
import numpy as np
from string import punctuation
from nltk.tokenize import sent_tokenize
morph = MorphAnalyzer()

     |████████████████████████████████| 51kB 1.6MB/s 
     |████████████████████████████████| 7.1MB 7.3MB/s 
     |████████████████████████████████| 256kB 52.9MB/s 
  Created wheel for DAWG: filename=DAWG-0.7.8-cp36-cp36m-linux_x86_64.whl size=781906 sha256=1eaa882d4da88a4752b86d2a5d512334786094db520ae5f905f3523e837050f5
  Stored in directory: /root/.cache/pip/wheels/d4/88/d0/4e4abc83eb8f59a71e8dbd8ba99fd5615a3af1fac1ef7f8825
Successfully built DAWG


# Задание 1.

In [3]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/besy_dostoevsky.txt
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/anna_karenina.txt
tolstoy = open('anna_karenina.txt').read()
dostoevsky = open('besy_dostoevsky.txt').read()

--2019-10-27 14:25:44--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/besy_dostoevsky.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2334469 (2.2M) [text/plain]
Saving to: ‘besy_dostoevsky.txt’

besy_dostoevsky.txt 100%[===================>]   2.23M  --.-KB/s    in 0.09s   

2019-10-27 14:25:44 (23.9 MB/s) - ‘besy_dostoevsky.txt’ saved [2334469/2334469]

--2019-10-27 14:25:46--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/anna_karenina.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting res

In [0]:
def normalize(text):
    normalized_text = [morph.parse(word.strip(punctuation))[0].normal_form for word in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

norm_tolstoy = normalize(tolstoy)
norm_dostoevsky = normalize(dostoevsky)

In [5]:
import nltk
nltk.download('punkt')
sentences_tolstoy = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(tolstoy)]
sentences_tolstoy = sentences_tolstoy[:1000]
sentences_dostoevsky = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dostoevsky)]
sentences_dostoevsky = sentences_dostoevsky[:1000]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
def ngrammer(tokens, n):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

Биграм

In [0]:
unigrams_tolstoy = Counter()
bigrams_tolstoy = Counter()

for sentence in sentences_tolstoy:
	unigrams_tolstoy.update(sentence)
	bigrams_tolstoy.update(ngrammer(sentence, 2))

unigrams_dostoevsky = Counter()
bigrams_dostoevsky = Counter()

for sentence in sentences_dostoevsky:
	unigrams_dostoevsky.update(sentence)
	bigrams_dostoevsky.update(ngrammer(sentence, 2))

матрикс

In [0]:
matrix_dostoevsky = np.zeros((len(unigrams_dostoevsky), len(unigrams_dostoevsky))) # 初始值0矩阵
id2word_dostoevsky = list(unigrams_dostoevsky) # 列表值为('word', count)
word2id_dostoevsky = {word:i for i, word in enumerate(id2word_dostoevsky)} # 字典值为('word', count):0（矩阵行和列定位）

for ngram in bigrams_dostoevsky: # 矩阵传值
    word1, word2 = ngram.split()
    matrix_dostoevsky[word2id_dostoevsky[word1]][word2id_dostoevsky[word2]] =  (bigrams_dostoevsky[ngram]/unigrams_dostoevsky[word1])

matrix_tolstoy = np.zeros((len(unigrams_tolstoy), len(unigrams_tolstoy)))

id2word_tolstoy = list(unigrams_tolstoy)
word2id_tolstoy = {word:i for i, word in enumerate(id2word_tolstoy)}

for ngram in bigrams_tolstoy:
    word1, word2 = ngram.split()
    matrix_tolstoy[word2id_tolstoy[word1]][word2id_tolstoy[word2]] =  (bigrams_tolstoy[ngram]/unigrams_tolstoy[word1])

генерация

In [0]:
def generate(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

In [20]:
print(generate(matrix_dostoevsky, id2word_dostoevsky, word2id_dostoevsky).replace('<end>', '\n'))

однажды в тринадцать год неутешный прасковья иван осип приходиться совершенно пред он безжалостно освистать так он с он за окно приглядываться к губернатор наш франт смотреть очень уж очень скоро начать обожание вопить теперь уважаю» но это быть в город от скворешник не мочь быть ещё с средство и частенько полёживать на чёрный кон и к он и не знать верно не захотеть тотчас же степан трофим отклонить предложение мамаша и степан трофим и это невинный привычка 
 губернатор как изволить почивать и запоминать такой тонкий и домашний анекдотец 
 видеть как бы и плакать 
 супруг весь они и


In [21]:
print(generate(matrix_tolstoy, id2word_tolstoy, word2id_tolstoy).replace('<end>', '\n'))

место начальник один 
 левин тотчас заметить изменение она одежда ни в ожидание когда в москва как хотеть слышать она 
 – подхватить степан аркадьй покачивать голова на знакомый игра она манер grande dame – и почему он не иметь взгляд оглядеть с отчаянный он это 
 это признак тот пора чтобы разъяснить недоразумение возникнуть между палец бумага из этот предмет который он быть ты приказать доложить 
 – ну разумеется – нешто выйти из совокупность весь готовый на современность и мастер кататься 
 он лицо он 
 левин войти к матерь 
 «там видно обойтись надо же женщины» –


триграмм

In [0]:
sentences_tolstoy_2 = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(tolstoy)]
sentences_tolstoy_2 = sentences_tolstoy_2[:1000]
sentences_dostoevsky_2 = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dostoevsky)]
sentences_dostoevsky_2 = sentences_dostoevsky_2[:1000]

In [0]:
unigrams_tolstoy_2 = Counter()
bigrams_tolstoy_2 = Counter()
trigrams_tolstoy_2 = Counter()

for sentence in sentences_tolstoy_2:
  unigrams_tolstoy_2.update(sentence)
  bigrams_tolstoy_2.update(ngrammer(sentence, 2))
  trigrams_tolstoy_2.update(ngrammer(sentence, 3))

unigrams_dostoevsky_2 = Counter()
bigrams_dostoevsky_2 = Counter()
trigrams_dostoevsky_2 = Counter()

for sentence in sentences_dostoevsky_2:
  unigrams_dostoevsky_2.update(sentence)
  bigrams_dostoevsky_2.update(ngrammer(sentence, 2))
  trigrams_dostoevsky_2.update(ngrammer(sentence, 3))

матрикс

In [0]:
matrix_dostoevsky_2 = np.zeros((len(bigrams_dostoevsky_2), len(unigrams_dostoevsky_2))) # 初始值0矩阵
id2word_dostoevsky_u = list(unigrams_dostoevsky_2) # 列表值为('word', count)
word2id_dostoevsky_u = {word:i for i, word in enumerate(id2word_dostoevsky_u)} # 字典值为('word', count):0（矩阵列定位）
id2word_dostoevsky_b = list(bigrams_dostoevsky_2) # 列表值为('word', count)
word2id_dostoevsky_b = {word:i for i, word in enumerate(id2word_dostoevsky_b)} # 字典值为('word', count):0（矩阵行定位）

for ngram in trigrams_dostoevsky_2: # 矩阵传值
  word1, word2, word3 = ngram.split()
  bigram = "{} {}".format(word1, word2)
  matrix_dostoevsky_2[word2id_dostoevsky_b[bigram]][word2id_dostoevsky_u[word3]] =  (trigrams_dostoevsky_2[ngram]/bigrams_dostoevsky_2[bigram])

In [0]:
matrix_tolstoy_2 = np.zeros((len(bigrams_tolstoy_2), len(unigrams_tolstoy_2)))
id2word_tolstoy_u = list(unigrams_tolstoy_2) # 列表值为('word', count)
word2id_tolstoy_u = {word:i for i, word in enumerate(id2word_tolstoy_u)} # 字典值为('word', count):0（矩阵列定位）
id2word_tolstoy_b = list(bigrams_tolstoy_2) # 列表值为('word', count)
word2id_tolstoy_b = {word:i for i, word in enumerate(id2word_tolstoy_b)} # 字典值为('word', count):0（矩阵行定位）

for ngram in trigrams_tolstoy_2:
    word1, word2, word3 = ngram.split()
    bigram = "{} {}".format(word1, word2)
    matrix_tolstoy_2[word2id_tolstoy_b[bigram]][word2id_tolstoy_u[word3]] =  (trigrams_tolstoy_2[ngram]/bigrams_tolstoy_2[bigram])

генерация

In [0]:
def generate_2(matrix, id2word_u, word2id_u, id2word_b, word2id_b, start = '<start> <start>', n=100):
    text = []
    current_idx = word2id_b[start]
    
    for i in range(n):
      chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
      text.append(id2word_u[chosen])
        
      start = start.split()[1] + ' ' + id2word_u[chosen]
        
      if id2word_u[chosen] == '<end>':
        start = '<start> <start>'
          
      current_idx = word2id_b[start]
    
    return ' '.join(text)

In [35]:
print(generate_2(matrix_dostoevsky_2, id2word_dostoevsky_u, word2id_dostoevsky_u, id2word_dostoevsky_b, word2id_dostoevsky_b).replace('<end>', '\n'))

степан трофим несколько раз и иногда после самый оживлённый и поэтический разговор они дружески расстаться горячо пожать друг друг в объятие не весь о один какой-нибудь домашний анекдотец 
 в один лишь излияние самый чувствительный любовь к он талантам» 
 но хроника она не иметь столько значение и влияние как в последний год стать член кружка 
 да н-н-ничего большой 
 но в удовлетворительный вид и если возможно войти в они 
 весь этот публика знать не знать как тут судить но вероятный что ничто не желать потому что ведь весь это она доставляться но у она быть несколько изящный


In [37]:
print(generate_2(matrix_tolstoy_2, id2word_tolstoy_u, word2id_tolstoy_u, id2word_tolstoy_b, word2id_tolstoy_b).replace('<end>', '\n'))

– а у я настоящее – так я иногда кажется 
 левин молчать поглядывать на степан аркадьй с салфетка и карточка в рука ожидать приказание 
 – да я непременно поехать – отвечать облонский 
 и то в этот таинственность совершаться 
 – я быть пьян… ну ты как поживать 
 когда он быть нужный так как краснеть взрослый человек – слегка сам тот не замечать но так как краснеть взрослый человек – слегка сам тот не замечать но так как краснеть мальчик – чувствовать что бояться он и он такой земной низменный существо что не удержаться 
 – я


Сравнение:

> биграм: 

1. Имеет более достоверную статистическую информацию, обладает больше верностью и практичностью;  
2.  Имеет меньше ограниченную информацию для следующего слова и обладает меньше различимостью.

> триграм: 

1.  Имеет больше ограниченную информацию для следующего слова и обладает больше различимостью;
2.  Нехватка длинности текста может привести к искажению вероятности; 
3.  занимает слишком великое пространство.


# Задание 2.

In [0]:
import re, gensim
from gensim.utils import tokenize
from gensim.summarization.textcleaner import split_sentences

In [0]:
text = '''Президент США сообщил, что правительство Турции проинформировало о прекращении боевых действий. По его мнению, перемирие на севере Сирии будет соблюдаться постоянно.

Господин Трамп назвал заслугой США перемирие, благодаря которому курды смогли спокойно покинуть границу с Турцией. «Этого добились мы, Соединенные Штаты, а не кто-то еще, никакая другая страна»,— сказал глава государства. По его словам, США удалось сохранить жизни многих курдов.

Дональд Трамп сообщил, что на территории Сирии останется небольшое число военных — «в районах, где есть нефть». «Мы будем ее защищать и решим, что делать с этим в будущем»,— сказал президент США.

Турция начала военную операцию «Источник мира» в Сирии 9 октября. В полночь 18 октября Анкара приостановила операцию. Ее завершили сегодня, 23 октября. Однако Турция не намерена полностью уходить с территории Сирии. В рамках договоренностей Вашингтон пообещал вывести курдские формирования за демаркационную линию в 30 км к югу от границы Сирии с Турцией. Россия и Турция начнут совместное патрулирование освобожденной курдами территории.
'''

In [0]:
sents_text = list(split_sentences(text))
print(sents_text)

['Президент США сообщил, что правительство Турции проинформировало о прекращении боевых действий.', 'По его мнению, перемирие на севере Сирии будет соблюдаться постоянно.', 'Господин Трамп назвал заслугой США перемирие, благодаря которому курды смогли спокойно покинуть границу с Турцией.', '«Этого добились мы, Соединенные Штаты, а не кто-то еще, никакая другая страна»,— сказал глава государства.', 'По его словам, США удалось сохранить жизни многих курдов.', 'Дональд Трамп сообщил, что на территории Сирии останется небольшое число военных — «в районах, где есть нефть».', '«Мы будем ее защищать и решим, что делать с этим в будущем»,— сказал президент США.', 'Турция начала военную операцию «Источник мира» в Сирии 9 октября.', 'В полночь 18 октября Анкара приостановила операцию.', 'Ее завершили сегодня, 23 октября.', 'Однако Турция не намерена полностью уходить с территории Сирии.', 'В рамках договоренностей Вашингтон пообещал вывести курдские формирования за демаркационную линию в 30 км к

In [0]:
symbols_sents = []
for sent in sents_text:
  symbols = []
  for smbl in sent:
    symbols.append(smbl)
  symbols_sents.append(symbols)
print(symbols_sents)

[['П', 'р', 'е', 'з', 'и', 'д', 'е', 'н', 'т', ' ', 'С', 'Ш', 'А', ' ', 'с', 'о', 'о', 'б', 'щ', 'и', 'л', ',', ' ', 'ч', 'т', 'о', ' ', 'п', 'р', 'а', 'в', 'и', 'т', 'е', 'л', 'ь', 'с', 'т', 'в', 'о', ' ', 'Т', 'у', 'р', 'ц', 'и', 'и', ' ', 'п', 'р', 'о', 'и', 'н', 'ф', 'о', 'р', 'м', 'и', 'р', 'о', 'в', 'а', 'л', 'о', ' ', 'о', ' ', 'п', 'р', 'е', 'к', 'р', 'а', 'щ', 'е', 'н', 'и', 'и', ' ', 'б', 'о', 'е', 'в', 'ы', 'х', ' ', 'д', 'е', 'й', 'с', 'т', 'в', 'и', 'й', '.'], ['П', 'о', ' ', 'е', 'г', 'о', ' ', 'м', 'н', 'е', 'н', 'и', 'ю', ',', ' ', 'п', 'е', 'р', 'е', 'м', 'и', 'р', 'и', 'е', ' ', 'н', 'а', ' ', 'с', 'е', 'в', 'е', 'р', 'е', ' ', 'С', 'и', 'р', 'и', 'и', ' ', 'б', 'у', 'д', 'е', 'т', ' ', 'с', 'о', 'б', 'л', 'ю', 'д', 'а', 'т', 'ь', 'с', 'я', ' ', 'п', 'о', 'с', 'т', 'о', 'я', 'н', 'н', 'о', '.'], ['Г', 'о', 'с', 'п', 'о', 'д', 'и', 'н', ' ', 'Т', 'р', 'а', 'м', 'п', ' ', 'н', 'а', 'з', 'в', 'а', 'л', ' ', 'з', 'а', 'с', 'л', 'у', 'г', 'о', 'й', ' ', 'С', 'Ш', 'А', ' ',

In [0]:
p = gensim.models.Phrases(symbols_sents, scoring='npmi', threshold=-1, min_count = 1)

In [0]:
p1 = gensim.models.Phrases(p[symbols_sents], scoring='npmi', threshold=-1, min_count = 1)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
p2 = gensim.models.Phrases(p1[p[symbols_sents]], scoring='npmi', threshold=-1, min_count = 1)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
p3 = gensim.models.Phrases(p2[p1[p[symbols_sents]]], scoring='npmi', threshold=-1, min_count = 1)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
list(p3[p2[p1[p[symbols_sents]]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


[['П_р_е_з_и_д_е_н_т_ _С_Ш_А_ _с_о',
  'о_б_щ_и_л_,_ _ч_т_о_ _п_р_а_в_и',
  'т_е_л_ь_с_т_в_о_ _Т_у_р_ц_и_и_ ',
  'п_р_о_и_н_ф_о_р_м_и_р_о_в_а_л_о',
  ' _о_ _п_р_е_к_р_а_щ_е_н_и_и_ _б',
  'о_е_в_ы_х_ _д_е_й_с_т_в_и_й_.'],
 ['П_о_ _е_г_о_ _м_н_е_н_и_ю_,_ _п',
  'е_р_е_м_и_р_и_е_ _н_а_ _с_е_в_е',
  'р_е_ _С_и_р_и_и_ _б_у_д_е_т_ _с',
  'о_б_л_ю_д_а_т_ь_с_я_ _п_о_с_т_о',
  'я_н_н_о_.'],
 ['Г_о_с_п_о_д_и_н_ _Т_р_а_м_п_ _н',
  'а_з_в_а_л_ _з_а_с_л_у_г_о_й_ _С',
  'Ш_А_ _п_е_р_е_м_и_р_и_е_,_ _б_л',
  'а_г_о_д_а_р_я_ _к_о_т_о_р_о_м_у',
  ' _к_у_р_д_ы_ _с_м_о_г_л_и_ _с_п',
  'о_к_о_й_н_о_ _п_о_к_и_н_у_т_ь_ ',
  'г_р_а_н_и_ц_у_ _с_ _Т_у_р_ц_и_е',
  'й_.'],
 ['«_Э_т_о_г_о_ _д_о_б_и_л_и_с_ь_ ',
  'м_ы_,_ _С_о_е_д_и_н_е_н_н_ы_е_ ',
  'Ш_т_а_т_ы_,_ _а_ _н_е_ _к_т_о_-',
  'т_о_ _е_щ_е_,_ _н_и_к_а_к_а_я_ ',
  'д_р_у_г_а_я_ _с_т_р_а_н_а_»_,_—',
  ' _с_к_а_з_а_л_ _г_л_а_в_а_ _г_о',
  'с_у_д_а_р_с_т_в_а_.'],
 ['П_о_ _е_г_о_ _с_л_о_в_а_м_,_ _С',
  'Ш_А_ _у_д_а_л_о_с_ь_ _с_о_х_р_а',
  'н_и_т_ь_ _ж_и_з_н_и_ 